In [1]:
import operator
from typing import Annotated, List, TypedDict, Union
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

### Agents
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
##

### Tools
from langchain_core.tools import tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool
###

from langgraph.constants import Send
from langgraph.graph import END, StateGraph, START

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
import os
from dotenv import load_dotenv
os.environ["LANGCHAIN_TRACING_V2"] = "false"
load_dotenv() ## ensure that your .env file in the same directory as this notebook has the OpenAI API Key

True

In [3]:
# tool utils
# tavily_tool = TavilySearchResults(max_results=5) # TODO: get TavilyAPIKey
python_repl_tool = PythonREPLTool()

@tool
def scrape_webpages(urls: List[str]) -> str:
    """Use requests and bs4 to scrape the provided web pages for detailed information."""
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return "\n\n".join(
        [
            f'\n{doc.page_content}\n'
            for doc in docs
        ]
    )

@tool
def get_telemetry(queries: List[str]) -> str: 
    """Use Alessandro's API to request samples of traces and/or metrics (gauges or counters) from prometheus and jaeger.
    @TODO: refine type signature for input to be a tuple of counted calls
    @TODO: implement the actual functionality
    """
    pass



In [4]:
# LLM setup 
llm = ChatOpenAI(model="gpt-4o")

In [5]:
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [6]:
gt_prompt = (
    "You are a telemeter tasked with using the provided tool to retrieve"
    " all requested metrics and traces by selecting and using the appropriate"
    " Jaeger and/or Prometheus API endpoints."
)

telemetry_retriever = create_agent(llm, [get_telemetry], gt_prompt)

In [7]:
anomdet_prompt = (
    "You are an expert at detecting anomalies in microservice application"
    " traces and metrics. Upon being provided a set of different metrics,"
    " as well as a description of an SLO violation, feel free to reason and"
    " or use the provided tools to determine if the reading is anomalous."
    " If you detect an anomaly, fill out the structured output. Otherwise,"
    " reply that there is no anomaly with the provided artifact."
)

anomaly_detector = create_agent(llm, [python_repl_tool], anomdet_prompt)

In [ ]:
class Anomaly(BaseModel):
    anomaly_occured: bool
    pass

class NonAnomaly(BaseModel):
    anomaly_occured: bool
    pass

AnomalyDetectorOut = Annotated[Union[Anomaly, NonAnomaly], Field(discriminator='anomaly_occured')]